<span style="font-size: 14pt">MIPT, Advanced ML, Spring 2018</span>

<span style="font-size: 16pt"> HW #5: Linear Models 

<span style="color:blue; font-size: 12pt">Alexey Romanenko </span>,
<span style="color:blue; font-size: 12pt; font-family: 'Verdana'"> alexromsput@gmail.com</span>


<h1 align="center">Organization Info</h1> 

* Дедлайн **11 апреля 2018 02:59** для всех групп.
* В качестве решения задания нужно прислать ноутбук с подробными комментариями (<span style='color:red'> без присланного решения результат контеста не будет засчитан </span>).
* <span style='color:red'>Название команды в контесте должно соответствовать шаблону: НомерГруппы_Имя_Фамилия, например, 594_Ivan_Ivanov</span>.

**Оформление дз**: 
- Присылайте выполненное задание на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2018_fall_<номер_группы>_<фамилия>``, к примеру -- ``ML2018_fall_495_ivanov``
- Выполненное дз сохраните в файл ``<фамилия>_<группа>_task<номер>.ipnb, к примеру`` -- ``ivanov_401_task5.ipnb``

**Вопросы**:
- Присылайте вопросы на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2018_fall Question <Содержание вопроса>``


--------
- **PS1:** Используются автоматические фильтры, и просто не найдем ваше дз, если вы неаккуратно его подпишите.
- **PS2:**  Просроченный дедлайн снижает максимальный вес задания по формуле, указнной на первом семинаре
- **PS3:** Допустимы исправление кода предложенного кода ниже, если вы считаете

<h1 align="center">Check Questions </h1> 

Ниже приводится список вопросов, с ответами на которые может быть полезно разобраться
для понимания темы.

**1.** Как выглядит решающее правило в линейной классификации? А зависимость, которой мы приближаем ответы в линейной регрессии?

**2.** Что такое функции потерь в задачах классификации и регрессии? Зачем они нужны?

**3.** Что такое регуляризаторы? Какими они бывают в задачах линейной классификации и регрессии? Зачем они нужны?

**4.** Как в общем виде выглядит оптимизационная задача в линейной классификации или линейной регрессии?

**5.** Как работает настройка весов в линейной модели с помощью SGD (Stochastic Gradient Decent)? Как выглядит правило обновления весов?

**6.** Учитывается ли коэффициент сдвига w0 в регуляризаторе? Почему?

**7.** Почему линейные модели рекомендуется применять к выборке с нормированными значениями признаков?

**8.** Как выглядит оптимизационная задача в логистической регрессии? А в SVM?

**9.** Выпишите и докажите формулу для весов в линейной регрессии (с квадратичной функцией потерь).
То же самое для гребневой регрессии.

**10.** Выпишите SGD для логистической регрессии с $l2$-регуляризацией и для SVM с линейным ядром.

**11.** В чем заключается идея ядер в SVM?

**12.** Какие преимущества и недостатки есть у линейных моделей?

**13.** На какие 3 матрицы раскладывается произвольная матрица при SVD? Что такое сингулярные числа

<h1 align="center">Contest 1 (40%)</h1> 
** Ссылка на контест: ** https://www.kaggle.com/t/68e2660e41d94d64a3c2251a1df16ab3

# Критерии оценивания
Для того чтобы получить оценку нужно
* 1) прислать код вашей модели (можно использовать только линейные модели), которая выдает один из финальных сабмиттов.
* 2) чтобы получить
    - хор (5)  и выше нужно превзойти результат benchmark_1;
    - отл (8) и выше нужно превзойти результат benchmark_2;
* 3) Ваше имя в лидерборде должно иметь вид <НомерГруппы>\_Имя\_Фамилия, например: 594\_Иван\_Иванов. Если имя не будет соответствовать формату организаторы вправе снизить оценку.

In [15]:
import re
import numpy as np
import pandas as pd

from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import cross_val_score, GridSearchCV, ShuffleSplit, StratifiedShuffleSplit

%matplotlib inline

In [28]:
def write_answer(predictions, file_name):
    with open(file_name, "w") as fout:
        fout.write("Id,Label\n")
        for i in range(predictions.shape[0]):
            fout.write(str(i+1)+","+str(predictions[i])+"\n")

In [2]:
data = pd.read_csv('./data/train.csv')
data.drop('id', axis = 1, inplace = True)
X_train = np.asarray(data[data.columns[range(1, data.shape[1])]], dtype = np.double)
Y_train = np.asarray(data[['label']], dtype = np.double).ravel()
data.head()

,label,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,feat9,...,feat207,feat208,feat209,feat210,feat211,feat212,feat213,feat214,feat215,feat216
0,1,7,0,3,0,2,3,0,6,0,...,3,4,2,2,0,13,0,11,1,3
1,1,0,11,0,0,10,1,0,0,4,...,0,2,0,0,2,8,1,13,0,4
2,0,9,0,3,0,1,3,0,4,0,...,48,11,2,0,0,4,0,2,0,0
3,0,0,9,3,2,25,0,4,0,0,...,1,14,1,0,0,0,3,0,17,1
4,0,0,0,0,0,2,5,0,0,0,...,3,12,0,3,0,4,0,24,4,0


In [3]:
data.groupby(['label']).count()

,feat1,feat2,feat3,feat4,feat5,feat6,feat7,feat8,feat9,feat10,...,feat207,feat208,feat209,feat210,feat211,feat212,feat213,feat214,feat215,feat216
label,,,,,,,,,,,,,,,,,,,,,
0,327,327,327,327,327,327,327,327,327,327,...,327,327,327,327,327,327,327,327,327,327
1,573,573,573,573,573,573,573,573,573,573,...,573,573,573,573,573,573,573,573,573,573


In [101]:
classifier = SVC()
cv = StratifiedShuffleSplit(n_splits = 5, test_size = 0.4)
parameters_grid = {
    'kernel' : ['poly'],
    'degree' : [1, 2, 3, 4],
    'gamma' : ['auto', 0.1, 0.5, 0.8],
    'coef0' : [0.0, 1.0],
}
grid_cv = GridSearchCV(classifier, parameters_grid, scoring = 'roc_auc', cv = 5, n_jobs=4)

In [102]:
%%time
grid_cv.fit(X_train, Y_train)

CPU times: user 1.75 s, sys: 96 ms, total: 1.84 s
Wall time: 19.4 s


GridSearchCV(cv=5, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'gamma': ['auto', 0.1, 0.5, 0.8], 'kernel': ['poly'], 'coef0': [0.0, 1.0], 'degree': [1, 2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [103]:
grid_cv.best_score_

0.9581921704873649

In [104]:
grid_cv.best_params_

{'coef0': 1.0, 'degree': 3, 'gamma': 'auto', 'kernel': 'poly'}

In [105]:
%%time
estimator = grid_cv.best_estimator_
estimator.fit(X_train, Y_train)
predictions = estimator.predict(X_test)

CPU times: user 1.17 s, sys: 0 ns, total: 1.17 s
Wall time: 1.17 s


In [106]:
write_answer(np.array(predictions, dtype=int), "./data/submission.csv")

In [36]:
classifier = LogisticRegression()
cv = StratifiedShuffleSplit(n_splits = 10, test_size = 0.2)
parameters_grid = {
    'penalty' : ['l1', 'l2'],
    'C' : [0.01, 0.1, 0.2, 1],
    'solver' : ['liblinear', 'saga']
}
grid_cv = GridSearchCV(classifier, parameters_grid, scoring = 'roc_auc', cv = cv, n_jobs=4)

In [37]:
%%time
grid_cv.fit(X_train, Y_train)

/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarni

/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarni

/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarni

CPU times: user 1.91 s, sys: 132 ms, total: 2.04 s
Wall time: 35.8 s


/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=None, test_size=0.2,
            train_size=None),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'C': [0.01, 0.1, 0.2, 1], 'solver': ['liblinear', 'saga'], 'penalty': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [38]:
grid_cv.best_score_

0.9294983277591974

In [39]:
%%time
estimator = grid_cv.best_estimator_
estimator.fit(X_train, Y_train)
predictions = estimator.predict(X_test)

CPU times: user 668 ms, sys: 12 ms, total: 680 ms
Wall time: 671 ms


/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [40]:
write_answer(np.array(predictions, dtype=int), "./data/submission.csv")

<h1 align="center">Contest 2 (40%)</h1> 
* Ссылка на контест: https://www.kaggle.com/t/2343d1882c6842258e3825b4a8a1e5a8

# Критерии оценивания
Для того чтобы получить оценку нужно
* 1) Прислать код вашей модели, которая выдает один из финальных сабмиттов.
* 2) чтобы получить
    - хор (5)  и выше нужно превзойти результат benchmark_1;
    - отл (8) и выше нужно превзойти результат benchmark_2;
* 3) Ваше имя в лидерборде должно иметь вид <НомерГруппы>\_Имя\_Фамилия, например: 594\_Иван\_Иванов. Если имя не будет соответствовать формату организаторы вправе снизить оценку.

In [1]:
import re
import numpy as np
import pandas as pd

from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing

from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import cross_val_score, GridSearchCV, ShuffleSplit, StratifiedShuffleSplit

%matplotlib inline

In [2]:
def write_answer2(id, predictions, file_name):
    with open(file_name, "w") as fout:
        fout.write("Id,sentiment\n")
        for i in range(predictions.shape[0]):
            fout.write(str(id[i])+","+str(predictions[i])+"\n")

In [3]:
train = pd.read_csv('./data1/train.csv')
X_train_df = train[['content']]
Y_train_df = train[['sentiment']]
train.head()

,Unnamed: 0,author,content,sentiment
0,26898,richardepryor,"@treasaint salad stuff, some chillis, whatever...",happiness
1,27635,reese,"@sunnyjamiel sunny, I'm a workin' on it. It's ...",neutral
2,3036,mutedriposte,@jolynnchew so early??,surprise
3,5604,sakizzie_1102,"So now, I have conjunctivitis in my left eye. ...",sadness
4,36111,poptrash,"Out and about in Deal, Kent. More sunshine req...",love


In [4]:
test = pd.read_csv('./data1/test.csv')
X_test_df = test[['content']]
test.head()

,id,author,content
0,32823,valicast,Good Morning
1,16298,btb103,I just put my computer up on craigslist. I've ...
2,28505,anavil,in ten minutes shopping demi lovato-back aro...
3,6689,ritwik1st,From twitterberry moved to ubertwitter - suffe...
4,26893,TightFreebies,@thriftymom TEAR*


In [5]:
def number_encode_features(init_df):
    result = init_df.copy()
    encoders = {}
    for column in result.columns:
        if result.dtypes[column] == np.object:
            encoders[column] = preprocessing.LabelEncoder()
            result[column] = encoders[column].fit_transform(result[column])
    return result, encoders

In [6]:
Y_train, encoders = number_encode_features(Y_train_df)

In [7]:
print(encoders)

{'sentiment': LabelEncoder()}


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [9]:
CountVectorizer()

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [12]:
count_vectorizer = CountVectorizer(min_df=5, ngram_range=(1, 5), stop_words='english') 
tf_idf = TfidfTransformer(use_idf=False)

In [13]:
sparse_feature_matrix = count_vectorizer.fit_transform(list(X_train_df.values.ravel()))
tf_idf.fit(sparse_feature_matrix)
sparse_feature_matrix = tf_idf.transform(sparse_feature_matrix)

In [14]:
sparse_feature_matrix

<30000x6793 sparse matrix of type '<class 'numpy.float64'>'
	with 190930 stored elements in Compressed Sparse Row format>

In [23]:
classifier = LogisticRegression()
cv = StratifiedShuffleSplit(n_splits = 10, test_size = 0.2)
parameters_grid = {
    'penalty' : ['l1', 'l2'],
    'C' : [0.01, 0.1, 0.2, 1],
    'solver' : ['liblinear', 'saga']
}
grid_cv = GridSearchCV(classifier, parameters_grid, scoring = 'accuracy', cv = cv, n_jobs=4)

In [24]:
%%time
grid_cv.fit(sparse_feature_matrix, Y_train.values.ravel())

/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarni

/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarni

CPU times: user 1min 47s, sys: 620 ms, total: 1min 47s
Wall time: 13min 47s


GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=None, test_size=0.2,
            train_size=None),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=4,
       param_grid={'C': [0.01, 0.1, 0.2, 1], 'solver': ['liblinear', 'saga'], 'penalty': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [25]:
grid_cv.best_score_

0.34946666666666665

In [32]:
grid_cv.best_params_

{'C': 1, 'penalty': 'l1', 'solver': 'saga'}

In [15]:
from sklearn.naive_bayes import MultinomialNB

In [16]:
estimators = [
    SGDClassifier(max_iter=5000, tol=0.1, penalty='l1', alpha=0.1),
    MultinomialNB(),
    SVC(kernel='rbf', C = 0.1, gamma=2),
    SVC(kernel='poly', C = 0.1),
    SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5),
    LinearSVC(penalty = 'l2', dual=False), 
    LogisticRegression(penalty='l1', C=0.2),
]

In [ ]:
cv_strategy = ShuffleSplit(n_splits=10, test_size=0.2)
for estimator in estimators:
    %time scoring = cross_val_score(estimator, sparse_feature_matrix, Y_train.values.ravel(), scoring='accuracy', cv=cv_strategy)
    print ('%s %.3lf' % (str(estimator).replace('(',' ').split()[0], scoring.mean()), '%.4lf' % scoring.std())

CPU times: user 4.75 s, sys: 16 ms, total: 4.77 s
Wall time: 4.77 s
SGDClassifier 0.215 0.0062
CPU times: user 576 ms, sys: 0 ns, total: 576 ms
Wall time: 651 ms
MultinomialNB 0.319 0.0061


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/IPython/core/magics/execution.py", line 1238, in time
    exec(code, glob, local_ns)
  File "<timed exec>", line 1, in <module>
  File "/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/model_selection/_validation.py", line 342, in cross_val_score
    pre_dispatch=pre_dispatch)
  File "/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/model_selection/_validation.py", line 206, in cross_validate
    for train, test in cv.split(X, y, groups))
  File "/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py", line 779, in __call__
    while self.dispatch_one_batch(iterator):
  File "/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py", line 625, in dispatch_one_batch
    self._dispatch(tasks)
  File "/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/externals/joblib/parallel.py", 

KeyboardInterrupt: 

SVC 0.319 0.0061


In [26]:
algo = LogisticRegression(penalty='l2', C=0.2),

In [13]:
from matplotlib import pyplot as plt
%matplotlib inline

In [27]:
algo.fit(sparse_feature_matrix, Y_train.values.ravel())

/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/linear_model/sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='saga', tol=0.0001,
          verbose=0, warm_start=False)

In [28]:
accuracy_score(algo.predict(sparse_feature_matrix), Y_train.values.ravel())

0.4081

In [29]:
Y_test = algo.predict(tf_idf.transform(count_vectorizer.transform(list(X_test_df.values.ravel()))))

In [30]:
print(encoders['sentiment'].inverse_transform(Y_test))

['happiness' 'neutral' 'neutral' ... 'sadness' 'worry' 'neutral']


/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [31]:
write_answer2(test['id'], encoders['sentiment'].inverse_transform(Y_test), "./data1/submission.csv")

/home/egorov/ml-mipt/myenv/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


<h1 align="center">Теоретические задачи (20%)</h1> 

** Задача 1**
Покажите, что регуляризатор в задаче линейной классификации имеет вероятностный смысл априорного распределения параметров моделей. Какие распределения задают 𝑙1-регуляризатор и 𝑙2-регуляризатор?

** Задача 2**
Покажите, что в случае линейно разделимой выборки функционал, максимизируемый при обучении логистической регрессии, достигает своего максимума в бесконечности (при бесконечно больших значениях компонент вектора весов), а значит для процесса обучения необходимо добавить регуляризационный член.

** Задача 3**
Задана выборка объектов, каждый из которых принадлежит одному из двух классов $Y = \{1; -1\}$. В некотором спрямляющем пространстве, задаваемом ядром $K_1(x_1; x_2)$, эта выборка является линейно разделимой, а в некотором спрямляющем пространстве, задаваемом ядром $K_2(x_1; x_2)$, не является линейно разделимой. Существует ли спрямляемое пространство, задаваемое ядром $K(x_1; x_2) = K_1(x_1; x_2) + K_2(x_1; x_2)$, в котором эта выборка будет линейно разделимой?

** Задача 4**
Покажите, что если предварительно центрировать выборку в задаче линейной регрессии, то параметр сдвига $𝑤_0$ (коэффициент перед константой) получится равным нулю.